# 01. FFT基礎

高速フーリエ変換（FFT）の基本概念を学び、時間領域から周波数領域への変換を理解します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq, fftshift
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. 単一周波数のFFT

In [ ]:
# パラメータ設定
sample_rate = 1000  # Hz
duration = 1.0      # 秒
frequency = 50      # Hz

# 時間軸の作成
t = np.linspace(0, duration, int(sample_rate * duration), False)

# サイン波の生成
signal = np.sin(2 * np.pi * frequency * t)

# FFTの実行
fft_result = fft(signal)
frequencies = fftfreq(len(signal), 1/sample_rate)

# 振幅スペクトルの計算
amplitude_spectrum = np.abs(fft_result)
phase_spectrum = np.angle(fft_result)

plt.figure(figsize=(15, 12))

# 元の信号
plt.subplot(3, 2, 1)
plt.plot(t[:200], signal[:200])  # 最初の0.2秒を表示
plt.title(f'元の信号（{frequency}Hz サイン波）')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.grid(True)

# FFT結果（全体）
plt.subplot(3, 2, 2)
plt.plot(frequencies, amplitude_spectrum)
plt.title('FFT振幅スペクトル（全体）')
plt.xlabel('周波数 (Hz)')
plt.ylabel('振幅')
plt.grid(True)

# FFT結果（正の周波数のみ）
plt.subplot(3, 2, 3)
positive_freq_mask = frequencies >= 0
plt.plot(frequencies[positive_freq_mask], amplitude_spectrum[positive_freq_mask], 'r-', linewidth=2)
plt.title('FFT振幅スペクトル（正の周波数）')
plt.xlabel('周波数 (Hz)')
plt.ylabel('振幅')
plt.grid(True)
plt.xlim(0, 100)

# 位相スペクトル
plt.subplot(3, 2, 4)
plt.plot(frequencies[positive_freq_mask], phase_spectrum[positive_freq_mask], 'g-', linewidth=2)
plt.title('FFT位相スペクトル')
plt.xlabel('周波数 (Hz)')
plt.ylabel('位相 (ラジアン)')
plt.grid(True)
plt.xlim(0, 100)

# dB表示
plt.subplot(3, 2, 5)
amplitude_db = 20 * np.log10(amplitude_spectrum[positive_freq_mask] + 1e-10)
plt.plot(frequencies[positive_freq_mask], amplitude_db, 'b-', linewidth=2)
plt.title('FFT振幅スペクトル（dB）')
plt.xlabel('周波数 (Hz)')
plt.ylabel('振幅 (dB)')
plt.grid(True)
plt.xlim(0, 100)

# ピーク周波数の検出
plt.subplot(3, 2, 6)
peak_idx = np.argmax(amplitude_spectrum[positive_freq_mask])
peak_freq = frequencies[positive_freq_mask][peak_idx]
peak_amp = amplitude_spectrum[positive_freq_mask][peak_idx]

plt.plot(frequencies[positive_freq_mask], amplitude_spectrum[positive_freq_mask], 'r-', linewidth=2)
plt.plot(peak_freq, peak_amp, 'ko', markersize=10, label=f'ピーク: {peak_freq:.1f} Hz')
plt.title('ピーク検出')
plt.xlabel('周波数 (Hz)')
plt.ylabel('振幅')
plt.legend()
plt.grid(True)
plt.xlim(0, 100)

plt.tight_layout()
plt.show()

print(f"検出されたピーク周波数: {peak_freq:.1f} Hz")
print(f"元の周波数: {frequency} Hz")
print(f"誤差: {abs(peak_freq - frequency):.1f} Hz")

## 2. 複数周波数成分のFFT

In [ ]:
# 複数の周波数成分を持つ信号
frequencies_input = [20, 50, 120]  # Hz
amplitudes = [1.0, 0.7, 0.5]

# 複合信号の生成
composite_signal = np.zeros_like(t)
individual_signals = []

for freq, amp in zip(frequencies_input, amplitudes):
    component = amp * np.sin(2 * np.pi * freq * t)
    individual_signals.append(component)
    composite_signal += component

# FFT実行
fft_composite = fft(composite_signal)
freq_axis = fftfreq(len(composite_signal), 1/sample_rate)
amplitude_composite = np.abs(fft_composite)

plt.figure(figsize=(15, 12))

# 個々の成分
plt.subplot(3, 2, 1)
colors = ['blue', 'red', 'green']
for i, (signal, freq, color) in enumerate(zip(individual_signals, frequencies_input, colors)):
    plt.plot(t[:200], signal[:200], color=color, alpha=0.7, 
            label=f'{freq} Hz', linewidth=2)
plt.title('個々の周波数成分')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.legend()
plt.grid(True)

# 合成信号
plt.subplot(3, 2, 2)
plt.plot(t[:200], composite_signal[:200], 'k-', linewidth=2)
plt.title('合成信号')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.grid(True)

# FFTスペクトル（線形スケール）
plt.subplot(3, 2, 3)
positive_mask = freq_axis >= 0
plt.plot(freq_axis[positive_mask], amplitude_composite[positive_mask], 'b-', linewidth=2)
plt.title('FFT振幅スペクトル')
plt.xlabel('周波数 (Hz)')
plt.ylabel('振幅')
plt.grid(True)
plt.xlim(0, 200)

# ピーク検出と注釈
plt.subplot(3, 2, 4)
plt.plot(freq_axis[positive_mask], amplitude_composite[positive_mask], 'b-', linewidth=2)

# ピークを検出して注釈
from scipy.signal import find_peaks
peaks, properties = find_peaks(amplitude_composite[positive_mask], 
                              height=np.max(amplitude_composite) * 0.1,
                              distance=10)

peak_freqs = freq_axis[positive_mask][peaks]
peak_amps = amplitude_composite[positive_mask][peaks]

plt.plot(peak_freqs, peak_amps, 'ro', markersize=8)
for freq, amp in zip(peak_freqs, peak_amps):
    plt.annotate(f'{freq:.1f} Hz', (freq, amp), 
                xytext=(5, 5), textcoords='offset points',
                fontsize=10, color='red')

plt.title('ピーク検出結果')
plt.xlabel('周波数 (Hz)')
plt.ylabel('振幅')
plt.grid(True)
plt.xlim(0, 200)

# dBスケール
plt.subplot(3, 2, 5)
amplitude_db = 20 * np.log10(amplitude_composite[positive_mask] + 1e-10)
plt.plot(freq_axis[positive_mask], amplitude_db, 'g-', linewidth=2)
plt.title('FFT振幅スペクトル（dB）')
plt.xlabel('周波数 (Hz)')
plt.ylabel('振幅 (dB)')
plt.grid(True)
plt.xlim(0, 200)

# 正規化スペクトル
plt.subplot(3, 2, 6)
normalized_amplitude = amplitude_composite[positive_mask] / np.max(amplitude_composite[positive_mask])
plt.plot(freq_axis[positive_mask], normalized_amplitude, 'm-', linewidth=2)
plt.title('正規化振幅スペクトル')
plt.xlabel('周波数 (Hz)')
plt.ylabel('正規化振幅')
plt.grid(True)
plt.xlim(0, 200)
plt.ylim(0, 1.1)

plt.tight_layout()
plt.show()

print("検出されたピーク周波数:")
for i, freq in enumerate(peak_freqs):
    print(f"ピーク {i+1}: {freq:.1f} Hz")

print("\n入力周波数:")
for i, freq in enumerate(frequencies_input):
    print(f"成分 {i+1}: {freq} Hz")

## 3. 窓関数の効果

In [ ]:
# 窓関数の効果を比較
def apply_window(signal, window_type):
    """
    信号に窓関数を適用
    """
    N = len(signal)
    if window_type == 'rectangular':
        window = np.ones(N)
    elif window_type == 'hanning':
        window = np.hanning(N)
    elif window_type == 'hamming':
        window = np.hamming(N)
    elif window_type == 'blackman':
        window = np.blackman(N)
    
    return signal * window, window

# テスト信号（周波数が非整数の場合のスペクトル漏れを観察）
test_freq = 50.5  # 非整数周波数
test_signal = np.sin(2 * np.pi * test_freq * t)

window_types = ['rectangular', 'hanning', 'hamming', 'blackman']

plt.figure(figsize=(16, 12))

for i, window_type in enumerate(window_types):
    # 窓関数を適用
    windowed_signal, window = apply_window(test_signal, window_type)
    
    # FFT実行
    fft_windowed = fft(windowed_signal)
    amplitude_windowed = np.abs(fft_windowed)
    
    # 窓関数の表示
    plt.subplot(4, 4, i*4 + 1)
    plt.plot(t[:200], window[:200], 'b-', linewidth=2)
    plt.title(f'{window_type.capitalize()}窓')
    plt.ylabel('振幅')
    plt.grid(True)
    
    # 窓適用後の信号
    plt.subplot(4, 4, i*4 + 2)
    plt.plot(t[:200], test_signal[:200], 'r--', alpha=0.5, label='元信号')
    plt.plot(t[:200], windowed_signal[:200], 'b-', linewidth=2, label='窓適用後')
    plt.title('窓適用後の信号')
    plt.ylabel('振幅')
    plt.legend()
    plt.grid(True)
    
    # FFTスペクトル（線形）
    plt.subplot(4, 4, i*4 + 3)
    plt.plot(freq_axis[positive_mask], amplitude_windowed[positive_mask], 'g-', linewidth=2)
    plt.title('FFTスペクトル')
    plt.ylabel('振幅')
    plt.grid(True)
    plt.xlim(30, 70)
    
    # FFTスペクトル（dB）
    plt.subplot(4, 4, i*4 + 4)
    amplitude_db = 20 * np.log10(amplitude_windowed[positive_mask] + 1e-10)
    plt.plot(freq_axis[positive_mask], amplitude_db, 'r-', linewidth=2)
    plt.title('FFTスペクトル (dB)')
    plt.ylabel('振幅 (dB)')
    plt.grid(True)
    plt.xlim(30, 70)
    
    if i == len(window_types) - 1:
        plt.xlabel('時間 (秒)')
        plt.subplot(4, 4, i*4 + 2)
        plt.xlabel('時間 (秒)')
        plt.subplot(4, 4, i*4 + 3)
        plt.xlabel('周波数 (Hz)')
        plt.subplot(4, 4, i*4 + 4)
        plt.xlabel('周波数 (Hz)')

plt.tight_layout()
plt.show()

print(f"テスト周波数: {test_freq} Hz")
print("各窓関数によるスペクトル漏れの特徴:")
print("- Rectangular窓: 最も鋭いピークだが、大きなサイドローブ")
print("- Hanning窓: サイドローブが小さく、一般的に使用される")
print("- Hamming窓: Hanningより少し鋭いピーク")
print("- Blackman窓: 最も小さなサイドローブだが、ピークが広い")

## 4. サンプリング定理とエイリアシング

In [ ]:
# サンプリング定理とエイリアシングの実演
def demonstrate_aliasing():
    # 高い周波数の信号
    high_freq = 150  # Hz
    duration = 1.0
    
    # 異なるサンプリングレートでの比較
    sample_rates = [100, 200, 400, 800]  # Hz
    
    plt.figure(figsize=(16, 12))
    
    for i, fs in enumerate(sample_rates):
        # 時間軸とサンプル点
        t_continuous = np.linspace(0, duration, 10000, False)  # 連続時間（参照用）
        t_sampled = np.linspace(0, duration, int(fs * duration), False)
        
        # 信号の生成
        signal_continuous = np.sin(2 * np.pi * high_freq * t_continuous)
        signal_sampled = np.sin(2 * np.pi * high_freq * t_sampled)
        
        # FFT
        fft_result = fft(signal_sampled)
        freqs = fftfreq(len(signal_sampled), 1/fs)
        amplitude = np.abs(fft_result)
        
        # 時間領域表示
        plt.subplot(4, 2, i*2 + 1)
        plt.plot(t_continuous[:500], signal_continuous[:500], 'b-', alpha=0.5, 
                label=f'連続信号 ({high_freq} Hz)')
        plt.plot(t_sampled[:int(0.05*fs)], signal_sampled[:int(0.05*fs)], 
                'ro-', markersize=4, label=f'サンプリング (fs={fs} Hz)')
        
        nyquist_freq = fs / 2
        plt.title(f'サンプリングレート: {fs} Hz (ナイキスト周波数: {nyquist_freq} Hz)')
        plt.ylabel('振幅')
        plt.legend()
        plt.grid(True)
        plt.xlim(0, 0.05)
        
        if fs < 2 * high_freq:
            aliased_freq = abs(high_freq % fs - fs) if high_freq % fs > fs/2 else high_freq % fs
            plt.text(0.02, 0.5, f'エイリアシング発生\n見かけの周波数: {aliased_freq:.1f} Hz', 
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="red", alpha=0.7),
                    fontsize=9)
        
        # 周波数領域表示
        plt.subplot(4, 2, i*2 + 2)
        positive_mask = freqs >= 0
        plt.plot(freqs[positive_mask], amplitude[positive_mask], 'g-', linewidth=2)
        plt.axvline(x=nyquist_freq, color='r', linestyle='--', 
                   label=f'ナイキスト周波数 ({nyquist_freq} Hz)')
        plt.title('FFTスペクトル')
        plt.ylabel('振幅')
        plt.legend()
        plt.grid(True)
        plt.xlim(0, fs/2 + 50)
        
        if i == len(sample_rates) - 1:
            plt.xlabel('時間 (秒)')
            plt.subplot(4, 2, i*2 + 2)
            plt.xlabel('周波数 (Hz)')
    
    plt.tight_layout()
    plt.show()

demonstrate_aliasing()

print("サンプリング定理:")
print("- 信号を正確に再構成するには、最高周波数の2倍以上のサンプリングレートが必要")
print("- ナイキスト周波数 = サンプリングレート / 2")
print("- サンプリング不足によりエイリアシング（折り返し雑音）が発生")

## 練習問題

1. 異なる位相を持つ同じ周波数の信号を合成し、位相が振幅スペクトルに与える影響を調べてみましょう
2. 白色ノイズのFFTスペクトルを計算し、その特徴を観察してみましょう
3. チャープ信号（周波数が時間とともに変化する信号）のFFTを計算し、固定周波数信号と比較してみましょう